<img src="https://github.com/Shubhwithai/GRE_Geometry_quiz/blob/main/Group%2042.png?raw=true" width="" height="50">

Educhain is a powerful Python package that leverages Generative AI to create engaging and personalized educational content. From generating multiple-choice questions to crafting comprehensive lesson plans, Educhain makes it easy to apply AI in various educational scenarios.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1JEcwCGfLM_4i3JJKHzMbYTP7KYUrRFyL?authuser=3#scrollTo=Je6BARehmNMK)




In [14]:
!pip install educhain

In [1]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GEMINI_KEY")

### Setting up the Custom LLM. Here Gemini model is used for generating the content ahead.

In [2]:
from educhain import Educhain, LLMConfig
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import os

gemini_flash = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=GOOGLE_API_KEY
)
flash_config = LLMConfig(custom_model=gemini_flash)
client = Educhain(flash_config)

### Core Functionality of the PYQ-to-Prep.
- Generating Mock Questions from popular exams such as NEET, JEE, Class 10, Class 12
- Generating explanation - doubt clearing from images
- Generating related questions from PDFs
- Generating questions related to a text block

In [8]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    """" Extracting Text from the provided PDF """
    reader = PdfReader(file_path)
    text = " ".join([page.extract_text() or "" for page in reader.pages])
    return " ".join(text.split())

def generate_questions_from_text(text, num_q=10, doubt=None):
    """ Generate Questions from the provided text """
    prompt = doubt if doubt else "Generate diverse questions from this PYQ"
    result = client.qna_engine.generate_questions_from_data(
        source=text,
        source_type="text",
        num=num_q,
        custom_instructions=(
            "Generate a mix of MCQs, True/False, Short and Long Answer questions based on this content. "
            "Add Bloom's taxonomy & difficulty levels where relevant."
        )
    )
    return result

def generate_mock_questions(exam_type, subject, topic="", num_q=10):
    """ Generate Mock - Practice question topic-wise using Educhain and Gemini """
    topic_query = f"{exam_type} {subject} {topic}".strip()
    result = client.qna_engine.generate_questions(
        topic=topic_query,
        num=num_q,
        custom_instructions=(
            "Generate diverse PYQ-style MCQ, TF, Short and Long answer questions with explanations, "
            "Bloom's levels, and difficulty rating."
        )
    )
    return result

def solve_image_doubt(image_path, prompt="Explain this image in detail"):
    """ Uses doubt_solver to explain the conept present in the provided image """
    explanation = client.qna_engine.solve_doubt(
        image_source=image_path,
        prompt=prompt,
        detail_level="High"
    )
    return explanation

### 📚 Use Case 1(a): Generate Mock Questions by Exam Type and Subject

##### Generating 'NEET' related questions
In the function **generate_mock_question** three attributes are passed:
- exam_type : Stating which popular exam we are targetting [Required]
- subject : Particular subject selection from the selected exam_type [Required]
- topic : Particular topic for questions focusing on the stated topic [Optional]
- num_q : Number of questions to be generated [Optional]

Here, Educhain's ***qna_engine.generate_questions*** model is being used.

In [10]:
mock = generate_mock_questions(exam_type="NEET", subject="Biology", topic="Photosynthesis", num_q=3)

for i, q in enumerate(mock.questions, 1):
    print(f"\nQ{i}: {q.question}")
    if hasattr(q, "options") and q.options:
        for j, opt in enumerate(q.options):
            print(f"  {chr(65 + j)}. {opt}")
        print(f"Answer: {q.answer}")
    else:
        print(f"Answer: {q.answer}")
    if getattr(q, "explanation", None):
        print(f"Explanation: {q.explanation}")


Q1: Which of the following is the primary function of the light-harvesting complex within the photosystems?
  A. To directly split water molecules
  B. To capture light energy and transfer it to the reaction center
  C. To synthesize ATP
  D. To fix carbon dioxide
Answer: To capture light energy and transfer it to the reaction center
Explanation: The light-harvesting complex contains pigment molecules that absorb light energy and transfer it to the reaction center, where the energy is used to excite electrons.

Q2: In the C4 pathway, the primary CO2 acceptor is:
  A. Ribulose-1,5-bisphosphate (RuBP)
  B. Phosphoenolpyruvate (PEP)
  C. Glyceraldehyde-3-phosphate (G3P)
  D. 3-phosphoglycerate (3-PGA)
Answer: Phosphoenolpyruvate (PEP)
Explanation: In C4 plants, CO2 is initially fixed in the mesophyll cells by the enzyme PEP carboxylase, which catalyzes the carboxylation of phosphoenolpyruvate (PEP) to form oxaloacetate.

Q3: Which of the following is NOT a product of the light-dependent 

### 📚 Use Case 1(b): Generate Mock Questions by Exam Type and Subject

In [13]:
mock = generate_mock_questions(exam_type="Class 12", subject="English", num_q=5)

for i, q in enumerate(mock.questions, 1):
    print(f"\nQ{i}: {q.question}")
    if hasattr(q, "options") and q.options:
        for j, opt in enumerate(q.options):
            print(f"  {chr(65 + j)}. {opt}")
        print(f"Answer: {q.answer}")
    else:
        print(f"Answer: {q.answer}")
    if getattr(q, "explanation", None):
        print(f"Explanation: {q.explanation}")


Q1: In 'The Last Lesson,' what did Franz regret not learning?
  A. German grammar
  B. His French grammar
  C. History
  D. Mathematics
Answer: His French grammar
Explanation: Franz regrets not taking his lessons seriously and wasting time instead of learning his native language.

Q2: What is the central theme of the poem 'Aunt Jennifer's Tigers'?
  A. The beauty of nature
  B. The constraints of married life on women
  C. The power of art
  D. The joys of embroidery
Answer: The constraints of married life on women
Explanation: The poem explores the oppression and lack of freedom experienced by women within the institution of marriage, using Aunt Jennifer's embroidered tigers as a symbol of freedom and power she lacks in her own life.

Q3: In 'Lost Spring', what does Saheb look for in the garbage dumps?
  A. Silver
  B. Toys
  C. Gold
  D. Clothes
Answer: Gold
Explanation: Saheb searches the garbage dumps hoping to find something valuable, metaphorically referred to as 'gold'.

Q4: Wh

### 🖼️ Use Case 2: Solve Doubts from Diagrams or Image Snapshots

##### Solving doubt or Generating explanation from Image Input
In the function **solve_image_doubt** two attributes are passed:
- image_path : Path to the image present in local environment [Required]
- prompt : A custom prompt stating what actions to be perfromed [Optional]

Here , Educhain's ***qna_engine.solve_doubt*** model is being used

In [11]:
img_path = "/content/note1.jpg" #Please do use your own Image to see the Result
explanation = solve_image_doubt(image_path=img_path, prompt="Explain the image in simple terms")
print(explanation)


explanation="The image describes the concept of a 'Thread' in computer science, particularly within the context of operating systems. It explains what a thread is, its components, benefits, and the different states a thread can be in during its lifecycle. The image also includes a state diagram illustrating the transitions between these thread states." steps=['Definition of Thread: A process is divided into multiple lightweight processes, each referred to as a thread.', 'Components of a Thread: Threads consist of a Program Counter (tracks the next instruction to execute), a Register Set (holds currently used variables), and a Stack (stores execution history).', 'Sharing Resources: Multiple threads can share the same address space, open files, and other resources.', 'Examples of Threads: Small processes like typing, formatting, or spell-checking can be executed by different threads.', 'Benefits of Threads:  Reduced creation time compared to processes.  Faster termination time compared t

### 📄 Use Case 3: Generate Questions from a PYQ PDF

##### Generating Questions from PDF
In the function **questions = generate_questions_from_text(text, num_q=5)** two attributes are passed:
- text : Providing the String from which questions are required to be generated.  [Required]
- num_q : No. of questions to be generated [Optional]

Here , Educhain's ***qna_engine.generate_questions_from_data*** model is being used

In [ ]:
pdf_path = "/content/IDB_AfterMidem notes _sai.pdf"
text = extract_text_from_pdf(pdf_path)
print(text)

questions = generate_questions_from_text(text, num_q=5)

for i, q in enumerate(questions.questions, 1):
    print(f"\nQ{i}: {q.question}")
    if hasattr(q, "options") and q.options:
        for j, opt in enumerate(q.options):
            print(f"  {chr(65 + j)}. {opt}")
        print(f"Answer: {q.answer}")
    else:
        print(f"Answer: {q.answer}")
    if getattr(q, "explanation", None):
        print(f"Explanation: {q.explanation}")

### ✍️ Use Case 4: Generate Questions from Custom Text Block

##### Generating Questions from Text
In the function **questions = generate_questions_from_text(text, num_q=5)** two attributes are passed:
- text : Providing the String from which questions are required to be generated.  [Required]
- num_q : No. of questions to be generated [Optional]

Here , Educhain's ***qna_engine.generate_questions_from_data*** model is being used

In [12]:
custom_text = """
Photosynthesis is a process used by plants and other organisms to convert light energy into chemical energy.
The chemical energy is stored in carbohydrate molecules, such as sugars, which are synthesized from carbon dioxide and water.
"""

result = generate_questions_from_text(custom_text, num_q=3)

for i, q in enumerate(result.questions, 1):
    print(f"\nQ{i}: {q.question}")
    if hasattr(q, "options") and q.options:
        for j, opt in enumerate(q.options):
            print(f"  {chr(65 + j)}. {opt}")
        print(f"Answer: {q.answer}")
    else:
        print(f"Answer: {q.answer}")
    if getattr(q, "explanation", None):
        print(f"Explanation: {q.explanation}")



Q1: What is the primary source of energy for photosynthesis?
  A. Chemical energy
  B. Heat energy
  C. Light energy
  D. Kinetic energy
Answer: Light energy
Explanation: Photosynthesis converts light energy into chemical energy.

Q2: What are the main products synthesized during photosynthesis?
  A. Proteins
  B. Lipids
  C. Carbohydrate molecules (sugars)
  D. Nucleic acids
Answer: Carbohydrate molecules (sugars)
Explanation: The chemical energy from photosynthesis is stored in carbohydrate molecules, such as sugars.

Q3: Which of the following are the reactants used in photosynthesis to produce sugars?
  A. Oxygen and glucose
  B. Nitrogen and methane
  C. Carbon dioxide and water
  D. Hydrogen and ammonia
Answer: Carbon dioxide and water
Explanation: Photosynthesis uses carbon dioxide and water to synthesize sugars.
